In [94]:
import pandas as pd
import os
import librosa  # python package for music and audio analysis
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import noisereduce as no
import sklearn
import warnings
warnings.filterwarnings('ignore')

In [25]:
classdit = pd.read_csv('BirdCLEF2020_Validation_Audio_and_Ground_Truth/species_names_to_codes.txt')

In [26]:
classaray = classdit.to_numpy()

In [27]:
classcodedict = dict(classaray)

In [46]:
classcodedict

{'Accipiter bicolor_Bicolored Hawk': 'bichaw1',
 "Accipiter cooperii_Cooper's Hawk": 'coohaw',
 'Accipiter gentilis_Northern Goshawk': 'norgos',
 'Accipiter nisus_Eurasian Sparrowhawk': 'eurspa1',
 'Accipiter poliogaster_Gray-bellied Hawk': 'gybhaw1',
 'Accipiter striatus_Sharp-shinned Hawk': 'shshaw',
 'Accipiter superciliosus_Tiny Hawk': 'tinhaw1',
 'Acrocephalus arundinaceus_Great Reed Warbler': 'grrwar1',
 'Acrocephalus scirpaceus_Eurasian Reed Warbler': 'eurwar1',
 'Actitis hypoleucos_Common Sandpiper': 'comsan',
 'Actitis macularius_Spotted Sandpiper': 'sposan',
 'Aechmophorus occidentalis_Western Grebe': 'wesgre',
 'Aegithalos caudatus_Long-tailed Tit': 'lottit1',
 'Aeronautes saxatalis_White-throated Swift': 'whtswi',
 'Agamia agami_Agami Heron': 'agaher1',
 'Agelaius phoeniceus_Red-winged Blackbird': 'rewbla',
 'Agelasticus cyanopus_Unicolored Blackbird': 'unibla2',
 'Agelasticus xanthophthalmus_Pale-eyed Blackbird': 'paebla2',
 'Aix galericulata_Mandarin Duck': 'manduc',
 'Ai

In [64]:
classes = []
birds = []
for root, dirs, files in os.walk("training sample/audio"):
    if root == 'training sample/audio':
        classes = dirs
        birds=dirs                
flist=[] 
blist=[] 
for i, bird in enumerate(birds):
    for root, dirs, files in os.walk("training sample/audio/"+bird):
        for file in files:
            if file.endswith(".mp3"):
                blist.append(os.path.join(root, file))
    flist.append(blist)
    blist=[]

In [66]:
len(birds)

20

In [68]:
len(flist)

20

In [71]:
classes

['Zonotrichia atricapilla_Golden-crowned Sparrow',
 'Volatinia jacarina_Blue-black Grassquit',
 'Xiphorhynchus ocellatus_Ocellated Woodcreeper',
 'Xanthocephalus xanthocephalus_Yellow-headed Blackbird',
 'Xiphorhynchus obsoletus_Striped Woodcreeper',
 'Xiphocolaptes promeropirhynchus_Strong-billed Woodcreeper',
 'Acanthis flammea_Common Redpoll',
 'Xenops minutus_Plain Xenops',
 'Willisornis poecilinotus_Common Scale-backed Antbird',
 'Xenops tenuirostris_Slender-billed Xenops',
 'Zebrilus undulatus_Zigzag Heron',
 'Piranga rubra_Summer Tanager',
 'Xiphorhynchus guttatus_Buff-throated Woodcreeper',
 'Zonotrichia albicollis_White-throated Sparrow',
 'Zonotrichia leucophrys_White-crowned Sparrow',
 'Xiphorhynchus elegans_Elegant Woodcreeper',
 'Glyphorynchus spirurus_Wedge-billed Woodcreeper',
 'Zimmerius gracilipes_Slender-footed Tyrannulet',
 'Zenaida macroura_Mourning Dove',
 'Xenops rutilans_Streaked Xenops']

In [72]:
ClassCode = []
for i in classes:
    if classcodedict.get(i) == None:
        ClassCode.append(i)
    else:
        ClassCode.append(classcodedict.get(i))

In [73]:
ClassCode

['gocspa',
 'blbgra1',
 'ocewoo1',
 'yehbla',
 'strwoo2',
 'stbwoo1',
 'Acanthis flammea_Common Redpoll',
 'plaxen1',
 'scbant3',
 'slbxen1',
 'zigher1',
 'sumtan',
 'butwoo1',
 'whtspa',
 'whcspa',
 'elewoo1',
 'webwoo1',
 'slftyr1',
 'moudov',
 'strxen1']

In [87]:
def saveMel(y, directory):
    N_FFT = 1024         # Number of frequency bins for Fast Fourier Transform
    HOP_SIZE = 1024      # Number of audio frames between STFT columns
    SR = 44100           # Sampling frequency
    N_MELS = 30          # Mel band parameters   
    WIN_SIZE = 1024      # number of samples in each STFT window
    WINDOW_TYPE = 'hann' # the windowin function
    FEATURE = 'mel'      # feature representation
                
    fig = plt.figure(1,frameon=False)
    fig.set_size_inches(6,6)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    librosa.display.specshow(
            librosa.core.amplitude_to_db(
                librosa.feature.mfcc(
                                dct_type=3,
                                y=y, 
                                sr=SR, 
                                n_fft=N_FFT, 
                                hop_length=HOP_SIZE, 
                                n_mels=N_MELS, 
                                htk=True, 
                                fmin=0.0, 
                                fmax=sr/2.0
                ), 
                    ref=1.0),
                sr=SR,
                hop_length=HOP_SIZE)
    fig.savefig(directory)
    fig.clear()
    ax.cla()
    plt.clf()
    plt.close('all')

In [90]:
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

In [88]:
def saveMel2(y, directory):

    N_FFT = 1024         # Number of frequency bins for Fast Fourier Transform
    HOP_SIZE = 1024      # Number of audio frames between STFT columns
    SR = 44100           # Sampling frequency
    N_MELS = 30          # Mel band parameters   
    WIN_SIZE = 1024      # number of samples in each STFT window
    WINDOW_TYPE = 'hann' # the windowin function
    FEATURE = 'mel'      # feature representation
    
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    frames = range(len(spectral_centroids))
    t = librosa.frames_to_time(frames)

    plt.rcParams['figure.figsize'] = (10,2)            
    fig = plt.figure(1,frameon=False)
    fig.set_size_inches(4,4)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    ax1 = plt.subplot(2, 1, 1)
    spectogram = librosa.display.specshow(
             librosa.core.amplitude_to_db(
                librosa.feature.melspectrogram(
                                y=y, 
                                sr=SR)),x_axis='time', y_axis='mel')
    plt.subplot(2, 1, 2, sharex=ax1)
    librosa.display.waveplot(y=y, sr=sr, alpha=0.4)
    plt.plot(t, normalize(spectral_centroids), color='r')
    
    
    fig.savefig(directory)
    fig.clear()
    ax.cla()
    plt.clf()
    plt.close('all')

In [81]:
size = {'desired': 10, # [seconds]
        'minimum': 5, # [seconds]
        'stride' : 0, # [seconds]
       }

In [95]:
step=1
if step>0:
    for bird, birdList in enumerate(flist):
         for birdnr, path in tqdm(enumerate(birdList)):
            directory="data/mels/"+ClassCode[bird]+"/"
            if not os.path.exists(directory):
                os.makedirs(directory)
            if not os.path.exists(directory+path.rsplit('/',1)[1].replace(' ', '')[:-4]+"1_1.png"):
                y, sr = librosa.load(path,mono=True)
                y = no.reduce_noise(audio_clip=y, noise_clip=y, verbose=False)  
                step = (size['desired']-size['stride'])*sr 
                nr=0;
                for start, end in zip(range(0,len(y),step),range(size['desired']*sr,len(y),step)):
                    nr=nr+1
                    if end-start > size['minimum']*sr:
                        melpath=path.rsplit('/',1)[1]
                        melpath=directory+melpath.replace(' ', '')[:-4]+str(nr)+"_"+str(nr)+".png"
                        saveMel2(y[start:end],melpath)                
            pass
else:    
    print("Error: Stride should be lower than desired length.")

In [ ]:
for birdnr, path in tqdm(enumerate(birdList)):
            directory="data/mels-19class2/"+str(bird)+birds50[bird][:size['name']]+"/"
            if not os.path.exists(directory):
                os.makedirs(directory)

In [43]:
# Plot spectogram
plt.figure(figsize=(10, 4))
D = librosa.amplitude_to_db(np.abs(librosa.stft(signal)), ref=np.max)
librosa.display.specshow(D, y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.title('Linear-frequency power spectrogram')
plt.show()

NameError: name 'signal' is not defined

<Figure size 720x288 with 0 Axes>